# 集成学习（ensemble learning）

## 集成学习的概念

集成学习通过构建并结合多个学习期来完成学习任务，有时也被称为多分类器系统、基于委员会的学习（committee-based learning）[1]

## 集成的方法

1.投票法，又叫统计法，即少数服从多数，用于预测任务

2.平均法，用于回归任务

3.堆叠法（stacking）

## 集成学习的分类

### Bagging类集成学习